In [2]:
"""
This code was written for the Week 1 Mini-Challenge for the
Space Situational Awareness and Artifical Intelligence Undergraduate Research
Opportunity program. Please abide by the license in the repository when utilizing
our code.
"""

# Libaries used are pandas, requests, and math
import pandas as pd
import requests
import math

#######################################
####### Challenge Information ##########
########################################
cols = ["ID","a","e","M","BigO","SmallO","i","MeanMotion"]
# Orbits in kilometers
LEO = [0,2000]
MEO = [2000,35800]
GEO = [35786,500]
HEO = [30000]

########################################


class TLEAnalyzer:
    def __init__(self, fileadr,permission):
        """Initiates TLEReader with a file address"""
        try:
            if permission not in ["a", "w", "r","a+","w+","r+"]:
                raise Exception("Invalid Permission. Valid open file\
                permissions include 'a', 'w', or 'r'. ")
        except:
            print("Something went wrong.")
        try:
            self.file = open(fileadr, permission)
        except:
            print("Something went wrong.")
        self.tleData = pd.DataFrame(columns=cols)

    def parse(self):
        """Adds TLE data to pandas dataframe"""
        lines = self.file.readlines()
        for x in range(int(len(lines)/2)):
            new_row = self.parse_line(lines[2*x], lines[2*x+1])
            self.tleData = pd.concat([new_row, self.tleData]).reset_index(drop = True)

    def parse_line(self, line1, line2):
        """Returns row of values for database"""
        pick1 = []
        pick2 = [0,4,6,3,0,2]
        arrParsed = []
        arr0 = line1.split()
        arr1 = line2.split()
        #Calculate semi major axis(in km) using formula below
        period = 1/float(arr1[7])*3600*24
        a = (period*398600/(4*3.14**2))**(1/3)
        arrParsed = pd.DataFrame({"ID":arr0[1],"a":a,"e":arr1[4],"M":arr1[6],"BigO":arr1[3],"SmallO":arr1[5],"i":arr1[2],"MeanMotion":arr1[7]},index=[0])
        return arrParsed

    def find_Satellite_Info(self, id):
        """Searches https://www.celestrak.com/satcat/search-results.php for satellite info"""
        session = requests.session()
        satInfo = self.tleData.loc[self.tleData["ID"] == id]

    def print(self):
        print(self.tleData)

    # Week 1 Challenge Functions
    def print_Satellite_Info(self):
        """Prints orbital information about each satellite"""
        for index, row in self.tleData.iterrows():
            print("-------------------------" + row[0] + "------------------------------")
            orbit = self.calculate_orbit(row)
            print("The satellite is in " + orbit)
            ecc = self.orbit_style(row)
            print("The satellite is in " + ecc)
            ang = self.crit_ang(row)
            print("The satellite is " + ang)
            sun = self.sun_sync(row)
            print("The satellie is " + sun)
            print("\n")

    def calculate_orbit(self,row):
        """Calculates which orbit a satellite is in"""
        MeanMotion = float(row[7])
        if float(row[6]) < .05 and MeanMotion - 1 <= .01:
            return "Geostationary Orbit"
        else:
            if MeanMotion > 11:
                return "Low Earth Orbit"
            elif MeanMotion < 11 and MeanMotion > 1:
                return "Medium Earth Orbit"
            else:
                return "Highly-Elliptical Orbit"
        return "an unknown orbit"

    def orbit_style (self,row):
        """Determins nature of orbit based on eccentricity"""
        ecc = float("."+row[2])
        if ecc <= .01:
            return "Circular Orbit"
        elif ecc <= .02:
            return "Near-Circular Orbit"
        else:
            return "Elliptical Orbit"

    def crit_ang(self, row):
        """Checks if satellite is in a critically inclined orbit"""
        ang = float(row[6])
        arg_perg = float(row[5])
        orb_per = 1 / float(row[7])
        a = float(row[1])
        if abs(ang - 63.4) <= 2:
            if abs(arg_perg - 270) <= 16 and abs(orb_per - .5) <= .1:
                return "in Molniya Orbit (Critically Inclined)"
            else:
                return "Critically Inclined"
        else:
            return "Not Critically Inclined"

    def sun_sync(self, row):
        """Determines whether or not a satellite in a non-elliptical orbit is in a Sun Synchronous orbit"""
        a = float(row[1])
        i = float(row[6])
        u = 5.167 * (10**12)
        if self.orbit_style(row) == "Circular Orbit"\
         or self.orbit_style(row) == "Near-Circular Orbit":
                T = 2 * math.pi * ((a**3) / u)** (.5)
                x = -(T/3.796)**(7/3)
                if abs(x - math.cos(i)) <= .1 * abs(math.cos(i)):
                    return "in Sun Synchronous Orbit"
                else:
                    return "in non Sun Synchronous Orbit"
        else:
            return "Elliptical N/A"


In [3]:
db = pd.read_csv("./Dataset1.csv")

In [8]:
db.head()

,NORAD_CAT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE
0,43,2019-10-09 05:37:44,15.264071,0.000426,35.4310,2.9892,293.2845,66.7430,6864.700,94.339,489.489,483.640
1,43,2019-10-09 16:37:02,15.264086,0.000424,35.4309,0.1103,296.5419,63.4870,6864.696,94.339,489.474,483.647
2,43,2019-10-10 13:01:27,15.264108,0.000421,35.4308,354.7638,302.6412,57.3905,6864.689,94.338,489.444,483.664
3,43,2019-10-11 12:34:14,15.264143,0.000416,35.4307,348.5946,309.8047,50.2310,6864.678,94.338,489.396,483.691
4,43,2019-10-11 15:42:36,15.264147,0.000415,35.4306,347.7721,310.7303,49.3060,6864.677,94.338,489.393,483.691


In [23]:
satellites = db["NORAD_CAT_ID"].unique()
satellite_dbs = {}
for satellite in satellites:
    satellite_dbs[satellite] = db.loc[db["NORAD_CAT_ID"]==satellite]
satellite_dbs[74415].head(20)


,NORAD_CAT_ID,EPOCH,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE
74700,74415,2019-10-08 23:48:00,0.062736,0.001585,97.754,94.212,299.495,176.549,6927.570697,5738.289,560.412794,538.454600
74701,74415,2019-10-09 08:59:00,0.062577,0.000214,97.748,94.583,176.320,208.668,6939.316678,5752.890,562.666154,559.693201
74702,74415,2019-10-09 16:54:00,0.062538,0.000292,97.746,94.911,332.650,35.807,6942.243220,5756.529,566.131890,562.080551
74703,74415,2019-10-09 22:42:00,0.062702,0.000845,97.753,95.147,19.760,215.380,6930.078497,5741.405,557.796579,546.086415
74704,74415,2019-10-10 09:51:00,0.062667,0.000677,97.751,95.606,352.100,234.956,6932.652968,5744.605,559.211664,549.820273
74705,74415,2019-10-10 14:40:00,0.062690,0.000769,97.752,95.805,10.915,221.256,6931.004789,5742.557,558.199163,547.536415
74706,74415,2019-10-11 00:14:00,0.062669,0.000671,97.751,96.199,353.763,233.623,6932.545160,5744.471,559.058568,549.757753
74707,74415,2019-10-11 07:48:00,0.062672,0.000894,97.751,96.521,319.112,172.807,6932.300782,5744.167,560.364044,547.963521
74708,74415,2019-10-11 15:24:00,0.062655,0.001020,97.751,96.824,203.983,200.206,6933.587074,5745.766,562.522300,548.377848
74709,74415,2019-10-11 23:11:00,0.062533,0.000459,97.745,97.150,308.142,49.443,6942.616461,5756.994,567.667392,561.291529


In [22]:
keys= satellite_dbs.keys()
satellite_dbs[74415].describe()

,NORAD_CAT_ID,MEAN_MOTION,ECCENTRICITY,INCLINATION,RA_OF_ASC_NODE,ARG_OF_PERICENTER,MEAN_ANOMALY,SEMIMAJOR_AXIS,PERIOD,APOGEE,PERIGEE
count,76.0,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000,76.000000
mean,74415.0,0.063041,0.003110,97.745487,106.591882,197.861921,188.499803,6906.651529,5712.656447,549.430127,507.598932
std,0.0,0.001253,0.007249,0.005738,7.303782,99.545891,86.805203,87.251433,107.232916,48.484152,132.395716
min,74415.0,0.062532,0.000045,97.731000,94.212000,9.542000,3.112000,6546.723326,5271.655000,259.455008,77.717644
25%,74415.0,0.062564,0.000558,97.741000,100.475250,122.858500,152.096750,6926.047199,5736.396500,557.974476,536.080680
50%,74415.0,0.062678,0.000826,97.746000,106.590000,202.768000,184.491500,6931.886565,5743.652500,560.388419,548.696196
75%,74415.0,0.062757,0.001576,97.750000,112.773000,288.555250,233.956250,6940.310267,5754.125000,565.051984,558.822359
max,74415.0,0.068290,0.032417,97.756000,119.382000,355.433000,358.014000,6942.633832,5757.015000,570.109780,563.652681


In [31]:
count = 0
lines = []
for key in satellite_dbs:
    count+=1
    if count < 20:
        lines.append(satellite_dbs[key].describe().iloc[2,:])
    print(count)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [33]:
print(len(satellite_dbs))

1717
